<a href="https://colab.research.google.com/github/GeryAdhane/Binary-Classification-using-keras-and-Deep-Learning-/blob/master/Gridworld_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Slving Gridworld Problem Using RL**

Gridworld is a common problem in reinforcement learning, where an agent has to navigate a rectangular grid of cells to reach a goal. The agent can only observe its current position and the reward it receives for each action. The agent has to learn a policy that maximizes the total reward over time.

# Method:
> We used Q-Learning, a model-free reinforcement learning (RL) algorithm, and Markov Decision Process (MDP) modeling to implement the RL-based agent. Q-Learning is a model-free, value-based, off-policy reinforcement learning algorithm that will find the best series of actions based on the agent's current state. MDP is a discrete-time stochastic control process. It provides a mathematical framework for modeling decision making in situations where outcomes are partly random and partly under the control of a decision maker.

> Q-Learning learns a Q-function, which is a table that maps each state-action pair to a value that represents the expected future reward of taking that action in that state. The Q-function is updated iteratively using the Bellman equation, which incorporates the immediate reward and the discounted future reward. The agent follows an exploration-exploitation trade-off strategy, such as epsilon-greedy, to balance between learning new information and exploiting the current knowledge. The Q-Learning algorithm converges to the optimal policy, which is the best action to take in each state, as long as all state-action pairs are visited infinitely often.

# Scenario:
> **Grid Layout**: Imagine an mxn grid where each cell represents a state the agent can be in.

> **Start and Goal**: The agent starts in one cell of the grid and the goal is to reach the designated goal.

> **Actions**: In each state, the agent can move in four directions: up, down, left, or right.

>**Boundaries**: Moving off the grid is not allowed and keeps the agent in the same state.

# Rewards:
> **Goal State**: Reaching the goal state gives a high positive reward (e.g., +10).

> **Other States**: Moving to any non-goal state gives a small negative reward (e.g., -1) to encourage efficiency.
#Objective:
> **Goal**: The agent's objective is to learn the best policy to reach the goal with maximum total reward.

> **Learning**: This is achieved through exploring the grid, receiving rewards, and updating a Q-table that represents the learned values of each action in each state.

In [10]:
#Import libraries
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

actions = {'U': 0, 'D': 1, 'L': 2, 'R': 3}


# Creating Interactive Visualization of the Gridworld environment

In [11]:

def animate_solution(grid_size, path):
    fig, ax = plt.subplots()

    # Set the aspect of the plot to be equal
    ax.set_aspect('equal')

    # Set the axes limits
    ax.set_xlim(0, grid_size[1])
    ax.set_ylim(0, grid_size[0])
    ax.axis('off')

    # Draw grid lines
    for x in range(grid_size[1] + 1):
        ax.axvline(x, color='black', linestyle='-', linewidth=1)
    for y in range(grid_size[0] + 1):
        ax.axhline(y, color='black', linestyle='-', linewidth=1)

    # Initial agent marker and path line
    agent, = ax.plot([], [], 'o', color='red', markersize=12, zorder=5)
    path_line, = ax.plot([], [], 'b-', linewidth=2, zorder=5)

    def init():
        agent.set_data([], [])
        path_line.set_data([], [])
        return (agent, path_line)

    def animate(i):
        if i < len(path):
            # Offset the x and y coordinates by 0.5 to center the agent in the cell
            agent.set_data(path[i][1] + 0.5, grid_size[0] - path[i][0] - 0.5)

            # Update the path
            path_x = [p[1] + 0.5 for p in path[:i+1]]
            path_y = [grid_size[0] - p[0] - 0.5 for p in path[:i+1]]
            path_line.set_data(path_x, path_y)
        return (agent, path_line)

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(path) + 1, interval=500, blit=True)
    plt.close(fig)  # Prevents the initial frame from showing inline below

    return anim

# Q-Learning Implementation

In [12]:
def choose_action(state, q_table, epsilon):
    if random.uniform(0, 1) < epsilon:
        return random.choice(list(actions.keys()))  # Explore: Random action
    else:
        return list(actions.keys())[np.argmax(q_table[state])]  # Exploit: Best action based on Q-table


def update_q_value(q_table, state, action, reward, next_state, alpha, gamma):
    current_q = q_table[state][action]
    max_future_q = np.max(q_table[next_state])
    new_q = (1 - alpha) * current_q + alpha * (reward + gamma * max_future_q)
    return new_q

def derive_policy(q_table):
    """Derive the policy from the Q-table."""
    policy = np.chararray(q_table.shape[:2], unicode=True)
    for i in range(q_table.shape[0]):
        for j in range(q_table.shape[1]):
            policy[i, j] = ['U', 'D', 'L', 'R'][np.argmax(q_table[(i, j)])]
    return policy

def train_q_learning(n, m, episodes, alpha, gamma, epsilon):
    actions = {'U': 0, 'D': 1, 'L': 2, 'R': 3}
    directions = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}
    goal_state = (n-1, m-1)
    q_table = np.zeros((n, m, len(actions)))
    total_rewards = []

    paths = []  # List to store paths for each episode

    for episode in range(episodes):
        state = (0, 0)
        done = False
        path = [state]  # Initialize path with the starting state

        while not done:
            action = choose_action(state, q_table, epsilon)
            next_state = (state[0] + directions[action][0], state[1] + directions[action][1])

            if 0 <= next_state[0] < n and 0 <= next_state[1] < m:
                if next_state == goal_state:
                    reward = 10
                    done = True
                else:
                    reward = -1
            else:
                next_state = state
                reward = -1

            action_index = actions[action]
            q_table[state][action_index] = update_q_value(q_table, state, action_index, reward, next_state, alpha, gamma)
            state = next_state

            path.append(next_state)  # Record each state in the path
            state = next_state

        paths.append(path) # Store the path for this episode

    return q_table, paths

# Main Function

In [13]:
# Parameters for Q-Learning
n = 8  # Grid height
m = 8  # Grid width
episodes = 10000
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
q_table, paths = train_q_learning(n, m, episodes, alpha, gamma, epsilon)

# Visualize the path of the last episode
last_episode_path = paths[-1]
anim = animate_solution((n, m), last_episode_path)
HTML(anim.to_jshtml())  # Display the animation

<ipython-input-11-05a29b62c99f>:30: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  agent.set_data(path[i][1] + 0.5, grid_size[0] - path[i][0] - 0.5)


# How to improve?

> **Add obstacles to some cells**: One possible name for the scenario where some cells are not possible to navigate, and cost of each cell varies, is a stochastic gridworld problem. This is a type of reinforcement learning problem, where an agent has to learn how to move in a grid environment with obstacles and different rewards. The agent does not know the transition probabilities or the reward function, and has to explore the grid and learn from its experience

>**Add UI to make the agent flexible - where to start and define the goal.**

> **Add UI to display each total cost and current cost.**
